In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from yaml import load
from yaml import CLoader as Loader
import re
import multiprocessing
import sys
from tqdm import tqdm
from collections import defaultdict

In [2]:
# Input
projectDir = Path("/home/_shared/jscliu/project/2025/Flagship/analysis/secondary/PGx/01d.genotype.manual")
resultsDir = projectDir/"results"

# Reference
star_allele_yml = projectDir/"script/resource/star_allele_variants.yml"
id_ref = "/home/_shared/jscliu/project/2025/Flagship/reference/sample_info_annot.2024-11-12.csv"
referenceDir_master = Path("/home/_shared/jscliu/project/2025/Flagship/reference/PharmGKB_ref")

gene_ls:list = ["CACNA1S", "ABCG2", "RYR1"]

# Output
summaryDir = projectDir/"summary"
summaryDir.mkdir(exist_ok=True)

# Read allele definitions as pd.DataFrame

In [3]:
with open(star_allele_yml) as handle:
    star_alleles:dict = load(handle, Loader=Loader)

In [4]:
# Process star_alleles for a ready-to-merge dataframe
column_names:list = ['gene', 'pharmgkb_allele', 'rsID', 'chr', 'start', 'end', 'allele_ref', 'allele_alt']
allele_ls:list = list()    # List of pd.Series of alleles
for gene, gene_info in star_alleles.items():
    if gene == "MT-RNR1":
        continue    # Skip MT-RNR1 as it will be handled separately
    gene_chr, gene_start, gene_end = re.split(":|-", gene_info['location'])
    for allele, mutations in gene_info['alleles'].items():
        if len(mutations['mutations']) == 0:
            pass
        else:
            for mut in mutations['mutations']:
                allele_entry = [gene, allele, mut[0], gene_chr, mut[1], mut[2], mut[3], mut[4]]
                allele_ls.append(pd.Series(allele_entry, index=column_names))

In [5]:
allele_def_df = pd.DataFrame(allele_ls)
allele_def_df['CPRA'] = allele_def_df.apply(lambda r: f"{r.chr}.{r.start}.{r.allele_ref}.{r.allele_alt}", axis=1)
allele_def_df.set_index('CPRA', inplace=True)
allele_def_df.head()

,gene,pharmgkb_allele,rsID,chr,start,end,allele_ref,allele_alt
CPRA,,,,,,,,
chr1.201060815.C.T,CACNA1S,c.520C>T,rs1800559,chr1,201060815,201060815,C,T
chr1.201091993.G.A,CACNA1S,c.3257G>A,rs772226819,chr1,201091993,201091993,G,A
chr4.88131171.G.T,ABCG2,rs2231142 variant (T),rs2231142,chr4,88131171,88131171,G,T
chr19.38433867.T.G,RYR1,c.38T>G,rs193922744,chr19,38433867,38433867,T,G
chr19.38440750.TGA.delTGA,RYR1,c.51_53del,rs193922745,chr19,38440750,38440752,TGA,delTGA


# Read id_ref as pd.DataFrame

In [10]:
# Read id reference and subset for useful columns
id_ref_df = pd.read_csv(id_ref, index_col=[0])
id_ref_df = id_ref_df.loc[id_ref_df['treated_ethnicity']=='Chinese', :].copy()    # Include Chinese only
id_ref_df =  id_ref_df[['founder_status']]
id_ref_df.head()

,founder_status
sre_participant_id,
SRE029440,Non-founder
SRE025838,Founder
SRE003784,Non-founder
SRE026979,Non-founder
SRE004787,Founder


# Read individual gene TSV as df

In [11]:
result_tsvs:list = list(resultsDir.glob("*genes.tsv"))

In [13]:
def read_tsv(tsv:Path)->pd.DataFrame:
    df = pd.read_table(tsv)
    df['CPRA'] = df.apply(lambda r: f"{r.CHROM}.{r.POS}.{r.REF}.{r.ALT}", axis=1)
    df['sample_id'] = tsv.name.split('.')[0]
    df.set_index('CPRA', inplace=True)
    return df

In [14]:
pool = multiprocessing.Pool(processes=10)    # Create Pool of 10 parallel processes max
results = pool.starmap(read_tsv, [(tsv,) for tsv in result_tsvs])    # If the function only takes one arguments
pool.close()    # Wait until all processes completed
pool.join()    # Join all output from processes
all_sample_gene_var_df = pd.concat(results)

# Call star alleles per sample

In [17]:
def call_genotype(paired_df:pd.DataFrame, gene:str)->str:
    """Call genotype based on the df with 
    individual variants & allele definition df."""
    paired_gene_df = paired_df.loc[paired_df['gene']==gene, :].copy()
    # paired_gene_df['GT'] = paired_gene_df['GT'].apply(lambda g: "/".join(sorted(re.split('\||/', g))))
    
    if len(paired_gene_df) == 0:
        # Return homozygous reference if no matched allele variant is found.
        genotype = "Reference/Reference"
    elif len(paired_gene_df) == 1:
        # Consider if there are multiple allele variants
        pharmgkb_allele = paired_gene_df['pharmgkb_allele'].tolist()[0]
        pharmgkb_allele_count = paired_gene_df['GT'].tolist()[0].count("1")
        if pharmgkb_allele_count == 1:
            genotype = f"Reference/{pharmgkb_allele}"
        elif pharmgkb_allele_count == 2:
            genotype = f"{pharmgkb_allele}/{pharmgkb_allele}"
        else:
            print(f"Error while understanding allele counts:\n{paired_gene_df}")
            sys.exit(1)
    else:
        genotype = np.nan

    return genotype

In [18]:
for sample_gene_df in tqdm(results):
    sample_id = sample_gene_df['sample_id'].unique().tolist()[0]
    out_genotype_tsv = resultsDir/f"{sample_id}.autosome_gene_manual.genotype.tsv"
    out_genotype_tsv.unlink(missing_ok= True)
    
    # Merge allele_def_df to sample_gene_df
    paired_df = sample_gene_df.merge(allele_def_df, left_index=True, right_index=True)

    CACNA1S_genotype, ABCG2_genotype, RYR1_genotype = [ call_genotype(paired_df, gene) for gene in gene_ls ]
    with open(out_genotype_tsv, 'w') as f:
        for gene in gene_ls:
            if globals()[f"{gene}_genotype"] == globals()[f"{gene}_genotype"]:
                f.write(f"{gene}\t{globals()[f'{gene}_genotype']}\n")    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24108/24108 [00:58<00:00, 413.37it/s]


# Summarize per sample findings

In [19]:
sample_gene_findings:list = list(resultsDir.glob("*.autosome_gene_manual.genotype.tsv"))
df_list:list = list()
for tsv in tqdm(sample_gene_findings):
    df = pd.read_table(tsv, names=['gene', 'genotype'])
    df['sre_participant_id'] = tsv.name.split('.')[0]
    df_list.append(df)
master_df = pd.concat(df_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24108/24108 [00:19<00:00, 1257.38it/s]


In [21]:
master_df.set_index('sre_participant_idid_ref_df', inplace=True)
master_df.head()

,gene,genotype
sre_participant_id,,
SRE027637,CACNA1S,Reference/Reference
SRE027637,ABCG2,Reference/rs2231142 variant (T)
SRE027637,RYR1,Reference/Reference
SRE027638,CACNA1S,Reference/Reference
SRE027638,ABCG2,Reference/Reference


In [25]:
per_gene_df = id_ref_df.merge(master_df, left_index=True, right_index=True, how='left')

# Drop na - out of project scope
per_gene_df.dropna(subset=['genotype'], inplace=True)

## Divide per_gene_df by genes, re-order columns, rename "Reference" (if needed), export to idv_geno_csv

In [27]:
def map_reference(genotype, gene)->str:
    map_d = {
        "CACNA1S": "Reference", 
        "ABCG2": "rs2231142 reference (G)",
        "RYR1": "Reference"
    }
    mapped_genotype = genotype.replace("Reference", map_d[gene])
    return mapped_genotype

per_gene_df['genotype'] = per_gene_df.apply(lambda r: map_reference(r.genotype, r.gene), axis=1)

In [28]:
per_gene_df['sre_participant_id'] = per_gene_df.index
per_gene_df.drop_duplicates(subset=['sre_participant_id', 'gene'], inplace=True)

In [30]:
reorder_colnames = ['founder_status', 'gene', 'sre_participant_id', 'genotype']
per_gene_df = per_gene_df.loc[:, reorder_colnames].copy()

In [31]:
# Label the allelel function of the pharmgkb_alleles
func_df_ls = list()
for gene in gene_ls:
    allele_func = referenceDir_master/f"{gene}/{gene}_allele_functionality_reference.csv"
    allele_func_df = pd.read_csv(allele_func, skiprows=1, index_col=[0]).loc[:, ['Allele Clinical Functional Status (Required)']]
    allele_func_df['gene'] = gene
    func_df_ls.append(allele_func_df)
allele_func_df = pd.concat(func_df_ls)
allele_def_df['allele_function'] = allele_def_df['pharmgkb_allele'].apply(
    lambda x: allele_func_df.loc[x, 'Allele Clinical Functional Status (Required)']
)

In [32]:
per_gene_df[['haplotype_1', 'haplotype_2']] = per_gene_df['genotype'].apply(lambda x: pd.Series(x.split('/')))

In [33]:
def map_phenotype(genotype, pheno_annot_df):
    q_genotype = f"{genotype.split('/')[1]}/{genotype.split('/')[0]}" if genotype not in pheno_annot_df.index else genotype
    phenotype = pheno_annot_df.loc[q_genotype, "Coded Diplotype/Phenotype Summary"]
    ehr = pheno_annot_df.loc[q_genotype, "EHR Priority Notation"]
    return pd.Series([phenotype, ehr])

In [34]:
for gene in gene_ls:
    all_idv_geno_csv = summaryDir/f"all_perGenes.{gene}.csv"
    idv_geno_csv = summaryDir/f"perGenes.{gene}.csv"
    allele_func = referenceDir_master/f"{gene}/{gene}_allele_functionality_reference.csv"
    pheno_annot = referenceDir_master/f"{gene}/{gene}_Diplotype_Phenotype_Table.csv"
    allele_func_map:dict = pd.read_csv(allele_func, skiprows=1, index_col=[0]).to_dict()['Allele Clinical Functional Status (Required)']
    pheno_annot_df:pd.DataFrame = pd.read_csv(pheno_annot, index_col=[0])
    # phenotype_map:dict = pd.read_csv(pheno_annot, index_col=[0]).to_dict()['Coded Diplotype/Phenotype Summary']
    # ehr_map:dict = pd.read_csv(pheno_annot, index_col=[0]).to_dict()['EHR Priority Notation']
    export_perGene_df = per_gene_df.loc[per_gene_df['gene']==gene, :].copy()
    export_perGene_df['haplotype_1_func'] = export_perGene_df['haplotype_1'].apply(lambda x: allele_func_map[x])
    export_perGene_df['haplotype_2_func'] = export_perGene_df['haplotype_2'].apply(lambda x: allele_func_map[x])
    export_perGene_df[['phenotype', 'EHR_priority_notation']] = export_perGene_df.apply(lambda r: map_phenotype(r.genotype, pheno_annot_df), axis=1)
    # export_perGene_df['phenotype'] = export_perGene_df['genotype'].apply(lambda x: phenotype_map[x])
    # export_perGene_df['EHR_priority_notation'] = export_perGene_df['genotype'].apply(lambda x: ehr_map[x])
    export_perGene_all_df = export_perGene_df.copy()
    export_perGene_df = export_perGene_df.loc[export_perGene_df.founder_status=="founder", :].copy()
    export_perGene_all_df.to_csv(all_idv_geno_csv, index=True, index_label='sre_patient_id')
    export_perGene_df.to_csv(idv_geno_csv, index=True, index_label='sre_patient_id')

# Calculate the allele frequencies

In [35]:
def count_zygosity(called_df):
    """Intake called_df, count the number of homozygous and heterozygous carriers into a dict"""
    homo_dict = defaultdict(int)
    hetero_dict = defaultdict(int)
    for _, row in called_df.iterrows():
        if row.haplotype_1 == row.haplotype_2:
            # Homozygous
            homo_dict[row.haplotype_1] += 1
        else:
            # Heterozygous
            hetero_dict[row.haplotype_1] += 1
            hetero_dict[row.haplotype_2] += 1
    return homo_dict, hetero_dict

In [36]:
for gene in gene_ls:
    af_csv = summaryDir/f"allele_frequency.{gene}.csv"
    df = per_gene_df.loc[(per_gene_df['gene']==gene) & (per_gene_df['founder_status']=='Founder'), :].copy()

    # Break genotype into haplotype
    df[['haplotype_1', 'haplotype_2']] = df['genotype'].apply(lambda g: pd.Series(g.split('/')))

    # Collapse tmp_df by haplotypes
    allele_df = pd.concat([
        df.loc[:, ['haplotype_1']].rename(columns={"haplotype_1": "haplotype"}), 
        df.loc[:, ['haplotype_2']].rename(columns={"haplotype_2": "haplotype"})
    ])
    allele_df['allele_count'] = 1
    
    # Calculate the allele frequency
    freq_df = allele_df.groupby('haplotype').agg("sum")
    freq_df['gene'] = gene
    freq_df['variant'] = freq_df.index
    freq_df['allele_number'] = freq_df['allele_count'].sum()
    freq_df.sort_values(by='allele_count', ascending=False, inplace=True)
    freq_df['allele_frequency'] = freq_df['allele_count'].apply(lambda c: c/freq_df['allele_count'].sum())

    # Annotate allele function
    allele_func = referenceDir_master/f"{gene}/{gene}_allele_functionality_reference.csv"
    allele_func_map:dict = pd.read_csv(allele_func, skiprows=1, index_col=[0]).to_dict()['Allele Clinical Functional Status (Required)']
    freq_df['allele_function'] = freq_df['variant'].apply(lambda x: allele_func_map[x])
    
    # Re-order columns
    freq_df = freq_df.loc[:, ['gene', 'variant', 'allele_function', 'allele_frequency', 'allele_count', 'allele_number']].copy()
    
    # Get zygosity and merge it to df
    homo_d, hetero_d = count_zygosity(df)
    homo_df = pd.DataFrame(homo_d, index=["no_homozygous_carriers"]).T
    hetero_df = pd.DataFrame(hetero_d, index=["no_heterozygous_carriers"]).T
    zygosity_df = hetero_df.merge(homo_df, left_index=True, right_index=True, how='outer').fillna(0).astype(int)
    freq_df = freq_df.merge(zygosity_df, left_index=True, right_index=True, how='left')
    
    # Export freq_df to CSV and save in af_df_d
    freq_df.to_csv(af_csv, index=False)

# Assign phenotypes and calculate the risk alleles carrier frequencies

In [38]:
keep_col = [
    'pharmacogene', 
    'genotype', 
    'phenotype', 
    'carrier_frequency', 
    'no_carriers_with_risk_alleles', 
    'all_genotyped_individuals'
]

for gene in gene_ls:
    phenotype_ref = referenceDir_master/f"{gene}/{gene}_Diplotype_Phenotype_Table.csv"
    pheno_csv = summaryDir/f"phenotype.{gene}.csv"
    df = per_gene_df.loc[(per_gene_df['gene']==gene) & (per_gene_df['founder_status']=='Founder'), :].copy()

    # Break genotype to sorted list and set_index
    df['diplotype'] = df['genotype'].apply(lambda x: "/".join(sorted(x.split('/'))))
    df.set_index('diplotype', inplace=True)
    
    # Read phenotype_ref as pd.DataFrame, break diplotype into sorted list and set_index
    phenotype_df = pd.read_csv(phenotype_ref)
    phenotype_df['diplotype'] = phenotype_df[f'{gene} Diplotype'].apply(lambda x: "/".join(sorted(x.split('/'))))
    phenotype_df.set_index('diplotype', inplace=True)
    
    # Merge phenotype_df to df
    annot_df = df.merge(phenotype_df, left_index=True, right_index=True, how='left')
    
    # Group annot_df by 'Coded Diplotype/Phenotype Summary', discard unwanted columns
    grouped_pheno_df = annot_df.groupby('Coded Diplotype/Phenotype Summary').agg(list)
    
    # Calculate the counts and frequencies
    grouped_pheno_df['pharmacogene'] = grouped_pheno_df['gene'].apply(lambda x: x[0])
    grouped_pheno_df['genotype'] = grouped_pheno_df['genotype'].apply(lambda x: ",".join(list(set(x))))
    grouped_pheno_df['phenotype'] = grouped_pheno_df.index
    grouped_pheno_df['no_carriers_with_risk_alleles'] = grouped_pheno_df['gene'].apply(len)
    grouped_pheno_df['all_genotyped_individuals'] = len(annot_df)
    grouped_pheno_df['carrier_frequency'] = grouped_pheno_df['no_carriers_with_risk_alleles'] / grouped_pheno_df['all_genotyped_individuals']
    
    # Subset column and export to pheno_csv
    out_df = grouped_pheno_df.loc[:, keep_col].copy()
    out_df.to_csv(pheno_csv, index=False)